Run the cell below to create your tear sheet.

In [ ]:
bt = get_backtest('5f5e2f99e524564684b7b562')
bt.create_full_tear_sheet()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import preprocessing

import statsmodels.api as sm
from statsmodels.tsa.stattools import coint

from scipy import stats
from quantopian.pipeline.factors import AnnualizedVolatility, Factor, Returns, CustomFactor
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume
from quantopian.pipeline.filters import  StaticAssets
from quantopian.pipeline.filters.morningstar import Q500US, Q1500US, Q3000US
from quantopian.pipeline.data.factset import EquityMetadata
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

In [ ]:
#prices = data.history(context.security_list,"close", 180, "1d")  

prices = USEquityPricing.close.latest

# Create pipeline
universe = Q1500US()
#universe = StaticAssets(symbols(['TSLA', 'AAPL']))

pipe = Pipeline(
        columns = {
            'price' : prices},
        screen = universe)

In [ ]:
pipe = Pipeline(
    columns = {'close_price': USEquityPricing.close.latest,
               'returns': Returns(inputs=[USEquityPricing.close], window_length=2),
               'log_returns': Returns(inputs=[USEquityPricing.close], window_length=2).log1p(),
              },
    screen = universe,
    )

result = run_pipeline(pipe, '2019-01-1', '2019-02-1')
result.index.rename(['date', 'stock'], inplace=True)
result.head()

In [ ]:
result['percent_change'] = result.groupby(level='stock').close_price.pct_change()
result['log_percent_chnge'] = np.log(1 + result.percent_change)


In [ ]:
result.head()

In [ ]:
k = 30
start = '2014-01-01'
end = '2015-01-01'
pricing = get_pricing('PEP', fields='price', start_date=start, end_date=end)
fundamentals = init_fundamentals()
num_shares = get_fundamentals(query(fundamentals.earnings_report.basic_average_shares,)
                              .filter(fundamentals.company_reference.primary_symbol == 'PEP',), end)
x = np.log(pricing)
v = x.diff()
m = get_pricing('PEP', fields='volume', start_date=start, end_date=end)/num_shares.values[0,0]

p0 = pd.rolling_sum(v, k)
p1 = pd.rolling_sum(m*v, k)
p2 = p1/pd.rolling_sum(m, k)
p3 = pd.rolling_mean(v, k)/pd.rolling_std(v, k)